<a href="https://colab.research.google.com/github/sriaryan18/RIKT-PROJECT/blob/main/Rikt_TF_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install transformers
! pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datasets import load_dataset
emotions=load_dataset('emotion')

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

model_name="bert-base-uncased"
tokenizer=AutoTokenizer.from_pretrained(model_name)


In [ ]:
def tokenize_function(example):
  return tokenizer(example['text'],truncation=True)

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [ ]:
tokenized_dataset=emotions.map(tokenize_function,batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 16000
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 2000
    })
})

In [ ]:
tf_train_dataset = tokenized_dataset["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)
tf_validation_dataset = tokenized_dataset["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["label"],
    shuffle=False,
    collate_fn=data_collator,
    batch_size=8,
)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


In [ ]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=6)

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from tensorflow.keras.optimizers.schedules import PolynomialDecay
import tensorflow as tf
batch_size = 8
num_epochs = 3

num_train_steps = len(tf_train_dataset) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=5e-5, end_learning_rate=0.0, decay_steps=num_train_steps
)
from tensorflow.keras.optimizers import Adam

opt = Adam(learning_rate=lr_scheduler)
# checkpoint_path='/content/drive/MyDrive/rikt/bhadresh_TF_Training/chkpts'
# cp_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_path, 
#     verbose=1, 
#     save_weights_only=True,
#     save_freq=20)


loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])
model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=10)

Epoch 1/10
2000/2000 [==============================] - 253s 117ms/step - loss: 0.3703 - accuracy: 0.8683 - val_loss: 0.1511 - val_accuracy: 0.9315
Epoch 2/10
2000/2000 [==============================] - 233s 116ms/step - loss: 0.1208 - accuracy: 0.9414 - val_loss: 0.1145 - val_accuracy: 0.9380
Epoch 3/10
2000/2000 [==============================] - 226s 113ms/step - loss: 0.0820 - accuracy: 0.9561 - val_loss: 0.1150 - val_accuracy: 0.9370
Epoch 4/10
2000/2000 [==============================] - 230s 115ms/step - loss: 0.0656 - accuracy: 0.9681 - val_loss: 0.1150 - val_accuracy: 0.9370
Epoch 5/10
2000/2000 [==============================] - 227s 113ms/step - loss: 0.0643 - accuracy: 0.9671 - val_loss: 0.1150 - val_accuracy: 0.9370
Epoch 6/10
2000/2000 [==============================] - 228s 114ms/step - loss: 0.0652 - accuracy: 0.9697 - val_loss: 0.1150 - val_accuracy: 0.9370
Epoch 7/10
2000/2000 [==============================] - 226s 113ms/step - loss: 0.0646 - accuracy: 0.9689 - val_

In [ ]:
model.save_pretrained('/content/drive/MyDrive/rikt/tf-bert-uncased1',)

In [ ]:
tokenizer.save_pretrained('/content/drive/MyDrive/rikt/tf-bert-uncased1')

('/content/drive/MyDrive/rikt/tf-bert-uncased1/tokenizer_config.json',
 '/content/drive/MyDrive/rikt/tf-bert-uncased1/special_tokens_map.json',
 '/content/drive/MyDrive/rikt/tf-bert-uncased1/vocab.txt',
 '/content/drive/MyDrive/rikt/tf-bert-uncased1/added_tokens.json',
 '/content/drive/MyDrive/rikt/tf-bert-uncased1/tokenizer.json')

In [ ]:
from transformers import pipeline
classifier = pipeline("text-classification",model='/content/drive/MyDrive/rikt/tf-bert-uncased2', return_all_scores=True,)



Some layers from the model checkpoint at /content/drive/MyDrive/rikt/tf-bert-uncased1 were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at /content/drive/MyDrive/rikt/tf-bert-uncased1.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.
/usr/local/lib/python

[[{'label': 'LABEL_0', 'score': 0.6624844670295715}, {'label': 'LABEL_1', 'score': 0.021030623465776443}, {'label': 'LABEL_2', 'score': 0.004234501160681248}, {'label': 'LABEL_3', 'score': 0.2999933362007141}, {'label': 'LABEL_4', 'score': 0.01118273101747036}, {'label': 'LABEL_5', 'score': 0.0010742672020569444}]]


In [ ]:
prediction = classifier('I am going to sue you' )
print(prediction)

[[{'label': 'LABEL_0', 'score': 0.014671724289655685}, {'label': 'LABEL_1', 'score': 0.006002183072268963}, {'label': 'LABEL_2', 'score': 0.0008389658178202808}, {'label': 'LABEL_3', 'score': 0.8721523284912109}, {'label': 'LABEL_4', 'score': 0.10524976998567581}, {'label': 'LABEL_5', 'score': 0.0010849862592294812}]]


In [ ]:
emotions['train'].features

{'text': Value(dtype='string', id=None),
 'label': ClassLabel(names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise'], id=None)}

In [ ]:
names=['sadness', 'joy', 'love', 'anger', 'fear', 'surprise']

In [ ]:
emotion_dict={}
for i in range(len(names)):
  emotion_dict[names[i]]="LABEL_"+str(i)

In [ ]:
emotion_dict

{'sadness': 'LABEL_0',
 'joy': 'LABEL_1',
 'love': 'LABEL_2',
 'anger': 'LABEL_3',
 'fear': 'LABEL_4',
 'surprise': 'LABEL_5'}

In [ ]:
label_to_emotion={}
for i,j in emotion_dict.items():
  label_to_emotion[j]=i

In [ ]:
label_to_emotion

{'LABEL_0': 'sadness',
 'LABEL_1': 'joy',
 'LABEL_2': 'love',
 'LABEL_3': 'anger',
 'LABEL_4': 'fear',
 'LABEL_5': 'surprise'}

In [ ]:
prediction

[[{'label': 'LABEL_0', 'score': 0.014671724289655685},
  {'label': 'LABEL_1', 'score': 0.006002183072268963},
  {'label': 'LABEL_2', 'score': 0.0008389658178202808},
  {'label': 'LABEL_3', 'score': 0.8721523284912109},
  {'label': 'LABEL_4', 'score': 0.10524976998567581},
  {'label': 'LABEL_5', 'score': 0.0010849862592294812}]]

In [ ]:
for i in prediction[0]:
  print(label_to_emotion[i['label']]+" score = "+str(i['score']))

sadness score = 0.014671724289655685
joy score = 0.006002183072268963
love score = 0.0008389658178202808
anger score = 0.8721523284912109
fear score = 0.10524976998567581
surprise score = 0.0010849862592294812


In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
    
Token: 
Add token as git credential? (Y/n) n
Token is valid.
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.huggingface/token
Login successful


In [ ]:
model.push_to_hub("tf_bert_uncased_emotion_detection")